In [ ]:
import numpy as np
import pandas as pd
from pyproj import Proj, transform

In [ ]:
# import numpy as np
# import pandas as pd

# # the center of the assaba (16.759763, -11.725705)
# lat_center = 16.759763  
# lon_center = -11.725705

# # Define pixel resolution in meters
# pixel_resolution = 500  # meters per pixel

# # Image shape (769 rows, 565 columns)
# rows, cols = 769, 565

# # Create empty lists to store coordinates
# latitudes = []
# longitudes = []

# # Loop through the image pixels
# for row in range(rows):
#     for col in range(cols):
#         # Calculate pixel offsets from the center
#         lat_offset = (row - rows // 2) * pixel_resolution
#         lon_offset = (col - cols // 2) * pixel_resolution
        
#         # Calculate real-world coordinates
#         latitude = lat_center + (lat_offset / 111320)  # approx conversion from meters to degrees
#         longitude = lon_center + (lon_offset / (40008000 / 360))  # convert meters to degrees for longitude

#         # Append to lists
#         latitudes.append(latitude)
#         longitudes.append(longitude)

# # Create a DataFrame to store the coordinates
# coords_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

# # reshape the DataFrame to match the image shape
# coords_df = coords_df.values.reshape(rows, cols, 2)  # Reshape to (769, 565, 2) for easier access

# # coordinate dict
# coords_dict = {(row, col): (coords_df[row, col, 0], coords_df[row, col, 1]) for row in range(rows) for col in range(cols)}


**Projection params ref: Ref_Doc/LCT_MCD12_User_Guide_V6.pdf**

**Page 3:**

Several parameters are needed to reproject the Sinusoidal HDF4 files to other projections using widely used software such as GDAL. 

Here we provide the values used for the upper left corner of the grid, the size of a single pixel, and the Sinusoidal projection string in Cartographic Projections Library (PROJ4) and Well-Known Text (WKT) formats.

- ULY Grid = 10007554.677, ULX Grid = -20015109.354
- Pixel Size (m) = 463.312716525
- Number of Pixels per Tile = 2400
- Projection Information
    - PROJ4: ‘+proj=sinu +a=6371007.181 +b=6371007.181 +units=m’
    - WKT:
    - PROJCS["Sinusoidal", GEOGCS["GCS_unnamed ellipse",
        DATUM["D_unknown", SPHEROID["Unknown",6371007.181,0]],
        PRIMEM["Greenwich",0], UNIT["Degree",0.017453292519943295]],
        PROJECTION["Sinusoidal"], PARAMETER["central_meridian",0],
        PARAMETER["false_easting",0], PARAMETER["false_northing",0],UNIT["Meter",1]

In [ ]:
# sinusoidal_projection.py

def gen_coords(lat_center, lon_center, pixel_resolution, rows, cols):
    """
    Generates a dictionary of geographic coordinates (latitude, longitude) for each pixel in a sinusoidal projection.

    The function calculates easting and northing values for an image's pixels based on the sinusoidal projection parameters.
    It then converts these projected coordinates back into latitude and longitude, returning a dictionary with 
    pixel indices as keys and geographic coordinates as values.

    Parameters:
    -----------
    lat_center: float
        Latitude of the center of the image (in degrees).
    lon_center: float
        Longitude of the center of the image (in degrees).
    pixel_resolution: float
        The pixel resolution of the image in meters.
    rows: int
        The number of rows (height) in the image.
    cols: int
        The number of columns (width) in the image.

    Returns:
    -----------
    coords_dict: dict
        A dictionary where the keys are pixel indices (row, col) and the values are tuples representing
        geographic coordinates (latitude, longitude) for the corresponding pixel.

    """
    
    from pyproj import Proj
    import pandas as pd

    # Initialize sinusoidal projection
    proj_sinu = Proj('+proj=sinu +a=6371007.181 +b=6371007.181 +units=m')

    # Calculate the easting and northing for the center point in the sinusoidal projection
    center_easting, center_northing = proj_sinu(lon_center, lat_center)

    # Create empty lists to store coordinates (easting and northing)
    eastings = []
    northings = []

    # Loop through the image pixels to calculate projected coordinates
    for row in range(rows):
        for col in range(cols):
            # Calculate pixel offsets from the center in meters
            northing_offset = (row - rows // 2) * pixel_resolution
            easting_offset = (col - cols // 2) * pixel_resolution

            # Calculate real-world coordinates in sinusoidal projection
            easting = center_easting + easting_offset
            northing = center_northing + northing_offset

            # Append to lists
            eastings.append(easting)
            northings.append(northing)

    # Convert projected coordinates back to longitude and latitude (pyproj returns lon, lat)
    longitudes, latitudes = proj_sinu(eastings, northings, inverse=True)

    # Create a DataFrame to store the coordinates
    coords_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

    # Reshape the DataFrame to match the image shape
    coords_df = coords_df.values.reshape(rows, cols, 2)  # Reshape to (rows, cols, 2)

    # Coordinate dictionary (row, col) -> (Latitude, Longitude)
    coords_dict = {(row, col): (coords_df[row, col, 0], coords_df[row, col, 1]) for row in range(rows) for col in range(cols)}

    return coords_dict